In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_data=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_data.head(3)

In [ ]:
print("Train dimension: ",train_data.shape)
print("Test dimension: ",test_data.shape)

In [ ]:
train_data.isnull().sum().sum()

In [ ]:
x_train=train_data.drop('label',axis=1)
y_train=train_data['label']
x_train=np.array(x_train)
y_train=np.array(y_train)

In [ ]:
train_plot = x_train.reshape(-1, 28, 28)

In [ ]:
plt.figure(figsize=(2, 2)) 
plt.imshow(train_plot[10], cmap='gray')

In [ ]:
sample=pd.read_csv("/kaggle/input/digit-recognizer/sample_submission.csv")
sample

In [ ]:
y_train = train_data['label'].values
x_train = train_data.drop(columns=['label']).values

y_test = np.array(sample['Label'])
x_test = np.array(test_data)

# Normalize
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Shuffle images and labels
x_train, y_train = shuffle(x_train, y_train, random_state=42)


print("Dimensions of training label:", y_train.shape)
print("Dimensions of training images:", x_train.shape)
print("Dimensions of the test label:", y_test.shape)
print("Dimensions of the test images:", x_test.shape)

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=10,
    zoom_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2 
)
datagen.fit(x_train)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(x_train,y_train)
print("X_train", X_train.shape)
print("X_val:", X_val.shape)
print("y_train:", y_train.shape)
print(" y_val:", y_val.shape)

In [ ]:
model = keras.Sequential([
    keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1, )),
    keras.layers.MaxPooling2D(pool_size=(2, 2)),
    keras.layers.Dropout(0.25),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation='softmax') 

])

model.compile(loss = "sparse_categorical_crossentropy", metrics=['accuracy'])

history = model.fit(
              X_train, 
              y_train, 
              epochs=100, 
              batch_size =128, 
              validation_split= 0.3,
              verbose=0
              )

In [ ]:
param_grid = { 
    'n_estimators': [200, 500],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']
}

In [ ]:
CV = GridSearchCV(estimator=model, param_grid=param_grid, cv= 5)
CV.fit(x_train, y_train)

In [ ]:
predictions = model.predict(x_test)
predicted_labels = np.argmax(predictions, axis=1)

# Create a DataFrame for the submission file
submission = pd.DataFrame({
    "ImageId": np.arange(1, len(predicted_labels) + 1),
    "Label": predicted_labels
})
submission.to_csv("submission.csv", index=False)

In [ ]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

In [ ]:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 4)) 

    # Accuracy graph
    ax1.plot(history.history['accuracy'], marker='o', linestyle='-')
    ax1.plot(history.history['val_accuracy'], marker='o', linestyle='-')
    ax1.set_title('Accuracy - Epochs', fontsize=16)
    ax1.set_xlabel('Epochs', fontsize=14)
    ax1.set_ylabel('Accuracy', fontsize=14, labelpad=10) 
    ax1.legend(['Train', 'Validation'], loc='lower right', fontsize=14)
    ax1.grid(True)

    # Loss graph
    ax2.plot(history.history['loss'], marker='o', linestyle='-')
    ax2.plot(history.history['val_loss'], marker='o', linestyle='-')
    ax2.set_title('Loss - Epochs', fontsize=16)
    ax2.set_xlabel('Epochs', fontsize=14)
    ax2.set_ylabel('Loss', fontsize=14, labelpad=10)  
    ax2.legend(['Train', 'Validation'], loc='upper right', fontsize=14, bbox_to_anchor=(1, 1))
    ax2.grid(True)

    plt.tight_layout()  
    plt.show() 